In [142]:
!pip install dendropy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.5/459.5 KB 4.9 MB/s eta 0:00:006.0 MB/s eta 0:00:01


In [1]:
import dendropy
from pandas import Series, DataFrame
import numpy as np

def make_distance_matrix(distances, sort=False):
    distances = distances.reset_index().pivot_table(index=0, columns=1, values='distance')
    distances.index.name=None
    distances.columns.name=None
    distances = distances.fillna(distances.T)

    missing_rows = [i for i in distances.index if i not in distances.columns]
    for row in missing_rows:
        distances[row] = distances.loc[row]

    missing_columns = [c for c in distances.columns if c not in distances.index]
    for column in missing_columns:
        distances.loc[column] = distances[column]
    
    if sort:
        distances = distances.sort_index().sort_index(axis=1)
    return distances.fillna(0)

def d(start, end, distances):
    return distances.loc[(start, end), 'distance']

def get_distance_dataframe(tree):
    pdm = tree.phylogenetic_distance_matrix()
    distances = [[*name, distance] for name, distance in zip(pdm.distinct_taxon_pair_iter(), pdm.distances())]
    distances = DataFrame(distances)
    
    def get_label(node):
        return node.label
    
    distances[0] = distances[0].map(get_label)
    distances[1] = distances[1].map(get_label)
    
    distances = distances.set_index([0, 1])
    distances = distances.rename(columns = {2:'distance'})

    return distances

def get_ratio(distances1, distances2):
    r1 = distances1.loc[common_leaves, common_leaves].sum().sum() / 2
    r2 = distances2.loc[common_leaves, common_leaves].sum().sum() / 2
    return r1/r2

def noisify_distances(distances):
    noise = np.triu(1 + np.random.random(distances.shape) / 10).round(2)
    noise += noise.T
    return distances * noise

In [9]:
tree = dendropy.Tree.get(path='trees/sym_tree_T2.txt', schema='newick') # or whatever relevant format if not newick

In [10]:
print(tree.as_ascii_plot())

                                                         /------------------- A
/--------------------------------------------------------+                     
|                                                        \------------------- D
+                                                                              
|                  /--------------------------------------------------------- B
\------------------+                                                           
                   |                  /-------------------------------------- F
                   \------------------+                                        
                                      |                  /------------------- E
                                      \------------------+                     
                                                         \------------------- G
                                                                               
                                        

In [2]:
tree1 = dendropy.Tree.get(path='test_tree.txt', schema='newick') # or whatever relevant format if not newick
distances1 = get_distance_dataframe(tree1)
distances1 = make_distance_matrix(distances1, True)
distances1 = noisify_distances(distances1)

tree2 = dendropy.Tree.get(path='test_tree_2.txt', schema='newick') # or whatever relevant format if not newick
distances2 = get_distance_dataframe(tree2)
distances2 = make_distance_matrix(distances2, True)

FileNotFoundError: [Errno 2] No such file or directory: 'test_tree.txt'

In [15]:
#common_leaves = ['A', 'C', 'G']
# 'B'->'X', 'D'->'R', 'F'->'W', 'H'->'Z'
common_leaves = [c for c in distances2.columns if c in distances1.columns]
epsilon = 0

In [16]:
def baseline_method(distances1, distances2, epsilon, common_leaves):
    # step 1 compute ratio    
    R = get_ratio(distances1, distances2)
    print('ratio:', R)
    
    # step 2 compute distances from candidate to common leaves
    candidates2 = [c for c in distances2.columns if c not in common_leaves]
    candidate_distances2 = distances2.loc[candidates2, common_leaves].sum(axis=1)
    
    candidates1 = [c for c in distances1.columns if c not in common_leaves]
    candidate_distances1 = distances1.loc[candidates1, common_leaves].sum(axis=1)
    
    # step 3 compare distances adjusted with the ratio
    candidate_distances2 = candidate_distances2 * R
    
    # step 4 find the closest matches
    matches = {}
    for candidate in candidates2:
        d_candidate = candidate_distances2[candidate]
        sorted_matches = (candidate_distances1 - d_candidate).abs().sort_values()        
        matches[candidate] = sorted_matches[sorted_matches <= sorted_matches.min() + epsilon].to_dict()

    return DataFrame(matches).stack().dropna().sort_values()

In [19]:
baseline_method(distances1, distances2, .5, common_leaves).sort_index()

ratio: 1.045294117647059


B  R    0.454824
   X    0.067824
D  R    0.007824
   X    0.514824
   Z    0.216882
E  W    0.111735
   Y    0.045059
F  W    0.071265
   Y    0.228059
H  R    0.305176
   Z    0.096118
dtype: float64

In [43]:
common_leaves

['A', 'C', 'G']

In [7]:
candidates1 = [c for c in distances1.columns if c not in common_leaves]
distances1.loc[candidates1, common_leaves].sum(axis=1).sort_values()

B    2.6650
D    3.0715
H    3.6410
E    4.6750
F    4.8375
dtype: float64

In [8]:
candidates2 = [c for c in distances2.columns if c not in common_leaves]
distances2.loc[candidates2, common_leaves].sum(axis=1).sort_values()

X    2.30
R    2.80
Z    3.00
Y    4.05
W    4.20
dtype: float64

In [34]:
# 'B'->'X', 'D'->'R', 'F'->'W', 'H'->'Z', 'E' -> 'Y'

In [ ]:
# Modify the tree to change the name of one node
# design a prototype method to pair nodes
# test prototype on modified tree